# Going Beyond the SimpleImputer() For Missing Data Imputation
## Learn to leverage model-based imputation
![](./images/unsplash.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@alxznder?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Alexander Lam</a>
        on 
        <a href='https://unsplash.com/s/photos/missing?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>

### Introduction

Despite the massive number of MOOCs and other online resources, there are still skill gaps in terms of dealing with certain data problems. One example is properly dealing with missing data in real-world datasets. Beginner learners might take this problem lightly and they are not to blame. Even though it is such a pressing issue, the complexity of data massingness problems has been greatly underestimated because of the availability of small, easy-to-work-with toy datasets. 

As a result, many beginner data scientists don't go beyond the simple techniques such as mean, median or mode imputation. Though these methods may suffice for simple datasets, they are in no way a competent solution to handling missing data with large datasets.

Like any other stage of data science workflow, missing data imputation is an iterative process. You should be able to use multiple methods and compare their results effectively. While the basic methods may end up performing well, it is rarely the case and you have to have a few backup strategies. 

This tutorial will introduce two more robust model-based imputation algorithms native to Sklearn - KNNImputer and IterativeImputer. Whereas for missing values in time-series data, you will learn about various interpolation techniques. Finally, we will see how to compare the results of different imputation techniques visually.